# Undersampling

In [ ]:
import pandas as pd
from sklearn.utils import resample

def create_undersampled_dataset(
    input_path: str = 'C://Users//a.karakuchukova//Desktop//Университет//Диплом//DataSets//dusha.crowd//dusha_crowd_processed.csv',
    output_path: str = 'C://Users//a.karakuchukova//Desktop//Университет//Диплом//DataSets//dusha.crowd//undersampled_dusha_crowd.csv',
    emotion_col: str = 'final_emotion',
    random_state: int = 42
):
    df = pd.read_csv(input_path)
    
    min_count = df[emotion_col].value_counts().min()
    print(f"Минимальный размер класса: {min_count} примеров")
    
    undersampled_frames = []
    for emotion, group in df.groupby(emotion_col):
        undersampled = resample(
            group,
            replace=False,
            n_samples=min_count,
            random_state=random_state
        )
        undersampled_frames.append(undersampled)
        print(f"Класс {emotion}: взято {len(undersampled)} примеров")
    
    df_under = pd.concat(undersampled_frames).reset_index(drop=True)
    
    df_under.to_csv(output_path, index=False)
    print(f"Undersampled dataset saved to {output_path}")

In [2]:
create_undersampled_dataset()

Минимальный размер класса: 11208 примеров
Класс angry: взято 11208 примеров
Класс neutral: взято 11208 примеров
Класс positive: взято 11208 примеров
Класс sad: взято 11208 примеров
Undersampled dataset saved to C://Users//a.karakuchukova//Desktop//Университет//Диплом//DataSets//dusha.crowd//undersampled_dusha_crowd.csv


In [3]:
df = pd.read_csv('C://Users//a.karakuchukova//Desktop//Университет//Диплом//DataSets//dusha.crowd//undersampled_dusha_crowd.csv')
df.head(2)

,Unnamed: 0,speaker_text,final_emotion
0,21666,выйди отсюда пожалуйста,angry
1,107358,убийственная внешность,angry


In [4]:
df.shape

(44832, 3)

In [5]:
df.final_emotion.value_counts(normalize=True)

final_emotion
angry       0.25
neutral     0.25
positive    0.25
sad         0.25
Name: proportion, dtype: float64

# Oversampling

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
from tqdm import tqdm
import os
import pandas as pd

In [ ]:
API_KEY=API_KEY

client = OpenAI(
    api_key=API_KEY,
    base_url="https://api.proxyapi.ru/openai/v1",
)
INPUT_CSV    = 'C://Users//a.karakuchukova//Desktop//Университет//Диплом//DataSets//dusha.crowd//dusha_crowd_processed.csv'
OUTPUT_CSV   = 'C://Users//a.karakuchukova//Desktop//Университет//Диплом//DataSets//dusha.crowd//gpt_oversampled_dusha_crowd.csv'
EMOTION    = 'final_emotion'
TEXT       = 'speaker_text'
MODEL      = 'gpt-4o-mini'

In [ ]:
GEN_PER_ITER = 5 
MAX_ITERS    = 10  
df = pd.read_csv(INPUT_CSV)
counts = df[EMOTION].value_counts()
target = counts.max()
print("Текущий максимум:", target)

new_rows = []

Текущий максимум: 87574


In [ ]:
for emotion, cnt in counts.items():
    need = target - cnt
    if need <= 0:
        continue

    print(f"\nКласс «{emotion}»: нужно {need} примеров, будем генерировать по {GEN_PER_ITER} шт за раз")
    few_shot = df[df[EMOTION]==emotion][TEXT].sample(3, random_state=42).tolist()
    total_iters = min(MAX_ITERS, (need + GEN_PER_ITER - 1)//GEN_PER_ITER)

    for it in range(total_iters):
        if need <= 0:
            break
        batch = min(GEN_PER_ITER, need)

        prompt = f"Ниже примеры, выражающие эмоцию «{emotion}»:\n"
        for ex in few_shot:
            prompt += f"- «{ex}»\n"
        prompt += (
            f"\nСгенерируй {batch} новых коротких реплик (≤15 слов), "
            "оформи их списком:\n"
        )
        for i in range(1, batch+1):
            prompt += f"{i}. \n"

        resp = client.chat.completions.create(
            model=MODEL,
            messages=[{"role":"user","content":prompt}],
        )
        text = resp.choices[0].message.content

        for line in text.splitlines():
            clean = line.lstrip('0123456789. ').strip(' «»"')
            if clean and need>0:
                new_rows.append({TEXT: clean, EMOTION: emotion})
                need -= 1

        print(f"  Итерация {it+1}/{total_iters}, осталось {need}")

        pd.concat([df, pd.DataFrame(new_rows)]).to_csv(OUTPUT_CSV, index=False)

    print(f"Готово по классу «{emotion}»: добавлено {counts[emotion] + (target-cnt - need) - cnt} примеров")

print("\nФинальные размеры классов:")
df_over = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
print(df_over[EMOTION].value_counts())
print(f"Промежуточный результат сохранён в {OUTPUT_CSV}")


Класс «sad»: нужно 69504 примеров, будем генерировать по 5 шт за раз
  Итерация 1/10, осталось 69499
  Итерация 2/10, осталось 69494
  Итерация 3/10, осталось 69489
  Итерация 4/10, осталось 69484
  Итерация 5/10, осталось 69479
  Итерация 6/10, осталось 69474
  Итерация 7/10, осталось 69469
  Итерация 8/10, осталось 69464
  Итерация 9/10, осталось 69459
  Итерация 10/10, осталось 69454
Готово по классу «sad»: добавлено 50 примеров

Класс «positive»: нужно 74473 примеров, будем генерировать по 5 шт за раз
  Итерация 1/10, осталось 74468
  Итерация 2/10, осталось 74463
  Итерация 3/10, осталось 74458
  Итерация 4/10, осталось 74453
  Итерация 5/10, осталось 74448
  Итерация 6/10, осталось 74443
  Итерация 7/10, осталось 74438
  Итерация 8/10, осталось 74433
  Итерация 9/10, осталось 74427
  Итерация 10/10, осталось 74422
Готово по классу «positive»: добавлено 51 примеров

Класс «angry»: нужно 76366 примеров, будем генерировать по 5 шт за раз
  Итерация 1/10, осталось 76361
  Итерация 2

In [ ]:
GEN_BATCH    = 10     
TARGET_PER_CLASS = 20000  
df_orig = df_over
counts  = df_orig[EMOTION].value_counts().to_dict()

print("Текущее распределение:", counts)
print("Будем генерить до", TARGET_PER_CLASS, "примеров на класс")

Текущее распределение: {'neutral': 87574, 'sad': 18120, 'positive': 13152, 'angry': 11259}
Будем генерить до 20000 примеров на класс


In [ ]:
new_rows = []

for emotion, cnt in counts.items():
    need = TARGET_PER_CLASS - cnt
    if need <= 0:
        continue

    few_shot = df_orig[df_orig[EMOTION] == emotion][TEXT] \
                  .sample(3, random_state=42).tolist()

    print(f"\nКласс «{emotion}»: нужно дополнить {need} примеров")
    pbar = tqdm(total=need, desc=emotion)

    while need > 0:
        batch = min(GEN_BATCH, need)

        prompt  = f"Ниже примеры высказываний, выражающих эмоцию «{emotion}» (на русском):\n"
        for txt in few_shot:
            prompt += f"- «{txt}»\n"
        prompt += (
            f"\nПожалуйста, сгенерируй {batch} новых коротких реплик "
            "(не более 15 слов каждая) с той же эмоцией и оформи их в виде списка:\n"
        )
        for i in range(1, batch+1):
            prompt += f"{i}. \n"

        resp = client.chat.completions.create(
            model=MODEL,
            messages=[{"role":"user","content":prompt}],
        )
        text = resp.choices[0].message.content

        for line in text.splitlines():
            clean = line.lstrip('0123456789. ').strip(' «»"')
            if clean and need > 0:
                new_rows.append({TEXT: clean, EMOTION: emotion})
                need -= 1
                pbar.update(1)
                if need == 0:
                    break

    pbar.close()


Класс «sad»: нужно дополнить 1880 примеров


sad: 100%|██████████| 1880/1880 [09:18<00:00,  3.37it/s]



Класс «positive»: нужно дополнить 6848 примеров


positive: 100%|██████████| 6848/6848 [44:37<00:00,  2.56it/s]   



Класс «angry»: нужно дополнить 8741 примеров


angry: 100%|██████████| 8741/8741 [54:31<00:00,  2.67it/s]   


In [26]:
df = pd.read_csv('C://Users//a.karakuchukova//Desktop//Университет//Диплом//DataSets//dusha.crowd//gpt_oversampled_dusha_crowd.csv')

In [27]:
df.head(3)

,Unnamed: 0,speaker_text,final_emotion
0,0.0,2drots литвин,neutral
1,2.0,2gis брюс ли,positive
2,3.0,2gis ка дыши,neutral


In [28]:
df.final_emotion.value_counts()

final_emotion
neutral     87574
sad         18120
positive    13152
angry       11259
Name: count, dtype: int64

In [ ]:
df_over = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
print(df_over[EMOTION].value_counts())

final_emotion
neutral     87574
positive    20000
sad         20000
angry       20000
Name: count, dtype: int64


In [31]:
len(df_over.speaker_text.unique())

133628

In [32]:
df_over.shape

(147574, 3)

In [35]:
df_over = df_over[['speaker_text','final_emotion']].drop_duplicates()

In [37]:
df_over.shape

(138462, 2)

In [38]:
len(df_over.speaker_text.unique())

133628

In [43]:
df_over.groupby('speaker_text').agg({'final_emotion':'count'}).reset_index()[df_over.groupby('speaker_text').agg({'final_emotion':'count'}).reset_index().final_emotion > 1]

,speaker_text,final_emotion
1,2gis брюс ли,2
21,a тупой что ли давай зеленоглазое такси,2
59,akula откуда она все знает,2
79,all hello,2
125,arkusha спокойной лучи,2
...,...,...
133480,ямщик гарик сукачев,2
133513,яндекс деньги,2
133541,яндекс такси правый руль,2
133600,ярославская еще летом,2


In [41]:
df_over[df_over['speaker_text'] == '2gis брюс ли']	

,speaker_text,final_emotion
1,2gis брюс ли,positive
114153,2gis брюс ли,neutral


In [ ]:
counts = df_over.groupby('speaker_text')['final_emotion'] \
                .nunique() \
                .reset_index(name='n_emotions')

good_texts = counts.loc[counts['n_emotions']==1, 'speaker_text']
df_clean = df_over[df_over['speaker_text'].isin(good_texts)].copy()


In [45]:
df_clean.shape

(128794, 2)

In [46]:
len(df_clean.speaker_text.unique())

128794

In [48]:
df_clean.tail(4)

,speaker_text,final_emotion
147567,Мне надоело это постоянное нытье!,angry
147568,Почему мне всегда приходится все решать?,angry
147570,"Хватит говорить, сделай уже что-то!",angry
147571,Я не буду это терпеть!,angry


In [49]:
df_clean.final_emotion.value_counts()

final_emotion
neutral     79973
sad         16654
positive    16568
angry       15599
Name: count, dtype: int64

In [50]:
df_clean.final_emotion.value_counts(normalize=True)

final_emotion
neutral     0.620937
sad         0.129307
positive    0.128640
angry       0.121116
Name: proportion, dtype: float64

In [51]:
df_clean.to_csv('C://Users//a.karakuchukova//Desktop//Университет//Диплом//DataSets//dusha.crowd//gpt_oversampled_dusha_crowd.csv')

In [ ]:
GEN_BATCH    = 10  
TARGET_PER_CLASS = 40000 
df_orig = df_clean
counts  = df_orig[EMOTION].value_counts().to_dict()

print("Текущее распределение:", counts)
print("Будем генерить до", TARGET_PER_CLASS, "примеров на класс")

Текущее распределение: {'neutral': 79973, 'sad': 16654, 'positive': 16568, 'angry': 15599}
Будем генерить до 40000 примеров на класс


In [ ]:
new_rows = []

for emotion, cnt in counts.items():
    need = TARGET_PER_CLASS - cnt
    if need <= 0:
        continue

    few_shot = df_orig[df_orig[EMOTION] == emotion][TEXT] \
                  .sample(3, random_state=42).tolist()

    print(f"\nКласс «{emotion}»: нужно дополнить {need} примеров")
    pbar = tqdm(total=need, desc=emotion)

    while need > 0:
        batch = min(GEN_BATCH, need)

        prompt  = f"Ниже примеры высказываний, выражающих эмоцию «{emotion}» (на русском):\n"
        for txt in few_shot:
            prompt += f"- «{txt}»\n"
        prompt += (
            f"\nПожалуйста, сгенерируй {batch} новых коротких реплик "
            "(не более 15 слов каждая) с той же эмоцией и оформи их в виде списка:\n"
        )
        for i in range(1, batch+1):
            prompt += f"{i}. \n"

        resp = client.chat.completions.create(
            model=MODEL,
            messages=[{"role":"user","content":prompt}],
        )
        text = resp.choices[0].message.content

        for line in text.splitlines():
            clean = line.lstrip('0123456789. ').strip(' «»"')
            if clean and need > 0:
                new_rows.append({TEXT: clean, EMOTION: emotion})
                need -= 1
                pbar.update(1)
                if need == 0:
                    break

    pbar.close()


Класс «sad»: нужно дополнить 23346 примеров


sad: 100%|██████████| 23346/23346 [2:26:05<00:00,  2.66it/s]   



Класс «positive»: нужно дополнить 23432 примеров


positive: 100%|██████████| 23432/23432 [2:18:44<00:00,  2.81it/s]  



Класс «angry»: нужно дополнить 24401 примеров


angry: 100%|██████████| 24401/24401 [3:35:05<00:00,  1.89it/s]  


In [72]:
df_new = pd.concat([df_clean, pd.DataFrame(new_rows)])

In [73]:
df_new.shape

(199973, 2)

In [ ]:
df_new.head(3)

,Unnamed: 0,speaker_text,final_emotion
0,0.0,2drots литвин,neutral
1,2.0,2gis брюс ли,positive
2,3.0,2gis ка дыши,neutral


In [75]:
df_new.final_emotion.value_counts()

final_emotion
neutral     79973
positive    40000
sad         40000
angry       40000
Name: count, dtype: int64

In [76]:
df_new.final_emotion.value_counts(normalize=True)

final_emotion
neutral     0.399919
positive    0.200027
sad         0.200027
angry       0.200027
Name: proportion, dtype: float64

In [77]:
len(df_new.speaker_text.unique())

176673

In [79]:
df_new = df_new.drop_duplicates()

In [80]:
df_new.groupby('speaker_text').agg({'final_emotion':'count'}).reset_index()[df_new.groupby('speaker_text').agg({'final_emotion':'count'}).reset_index().final_emotion > 1]


,speaker_text,final_emotion
33872,Почему всё всегда так сложно?,2
33884,Почему всё должно быть так сложно?,2
58592,Этот момент навсегда останется в моем сердце.,2


In [81]:
df_new[df_new.speaker_text == 'Этот момент навсегда останется в моем сердце.']

,speaker_text,final_emotion
134980,Этот момент навсегда останется в моем сердце.,positive
848,Этот момент навсегда останется в моем сердце.,sad


In [ ]:
counts = df_new.groupby('speaker_text')['final_emotion'] \
                .nunique() \
                .reset_index(name='n_emotions')

good_texts = counts.loc[counts['n_emotions']==1, 'speaker_text']
df_clean = df_new[df_new['speaker_text'].isin(good_texts)].copy()

In [83]:
df_clean.final_emotion.value_counts()

final_emotion
neutral     79973
angry       34163
sad         31954
positive    30580
Name: count, dtype: int64

In [84]:
df_clean.final_emotion.value_counts(normalize=True)

final_emotion
neutral     0.452669
angry       0.193372
sad         0.180868
positive    0.173091
Name: proportion, dtype: float64

In [85]:
df_clean.to_csv('C://Users//a.karakuchukova//Desktop//Университет//Диплом//DataSets//dusha.crowd//gpt_oversampled_dusha_crowd.csv')

In [86]:
df_clean.shape

(176670, 2)

In [ ]:
GEN_BATCH    = 10    
TARGET_PER_CLASS = 35000 
df_orig = df_clean
counts  = df_orig[EMOTION].value_counts().to_dict()

print("Текущее распределение:", counts)
print("Будем генерить до", TARGET_PER_CLASS, "примеров на класс")

Текущее распределение: {'neutral': 79973, 'angry': 34163, 'sad': 31954, 'positive': 30580}
Будем генерить до 35000 примеров на класс


In [ ]:
new_rows = []

for emotion, cnt in counts.items():
    need = TARGET_PER_CLASS - cnt
    if need <= 0:
        continue

    few_shot = df_orig[df_orig[EMOTION] == emotion][TEXT] \
                  .sample(3, random_state=42).tolist()

    print(f"\nКласс «{emotion}»: нужно дополнить {need} примеров")
    pbar = tqdm(total=need, desc=emotion)

    while need > 0:
        batch = min(GEN_BATCH, need)

        prompt  = f"Ниже примеры высказываний, выражающих эмоцию «{emotion}» (на русском):\n"
        for txt in few_shot:
            prompt += f"- «{txt}»\n"
        prompt += (
            f"\nПожалуйста, сгенерируй {batch} новых коротких реплик "
            "(не более 15 слов каждая) с той же эмоцией и оформи их в виде списка:\n"
        )
        for i in range(1, batch+1):
            prompt += f"{i}. \n"

        resp = client.chat.completions.create(
            model=MODEL,
            messages=[{"role":"user","content":prompt}],
        )
        text = resp.choices[0].message.content

        for line in text.splitlines():
            clean = line.lstrip('0123456789. ').strip(' «»"')
            if clean and need > 0:
                new_rows.append({TEXT: clean, EMOTION: emotion})
                need -= 1
                pbar.update(1)
                if need == 0:
                    break

    pbar.close()


Класс «angry»: нужно дополнить 837 примеров


angry: 100%|██████████| 837/837 [06:19<00:00,  2.21it/s]



Класс «sad»: нужно дополнить 3046 примеров


sad: 100%|██████████| 3046/3046 [22:00<00:00,  2.31it/s]



Класс «positive»: нужно дополнить 4420 примеров


positive: 100%|██████████| 4420/4420 [34:56<00:00,  2.11it/s]


In [103]:
df_new = pd.concat([df_clean, pd.DataFrame(new_rows)])

In [ ]:
counts = df_new.groupby('speaker_text')['final_emotion'] \
                .nunique() \
                .reset_index(name='n_emotions')

good_texts = counts.loc[counts['n_emotions']==1, 'speaker_text']
df_clean = df_new[df_new['speaker_text'].isin(good_texts)].copy()

In [110]:
df_clean.shape

(184973, 2)

In [106]:
df_clean.final_emotion.value_counts()

final_emotion
neutral     79973
positive    35000
sad         35000
angry       35000
Name: count, dtype: int64

In [107]:
df_clean.final_emotion.value_counts(normalize=True)

final_emotion
neutral     0.432350
positive    0.189217
sad         0.189217
angry       0.189217
Name: proportion, dtype: float64

In [109]:
len(df_clean.speaker_text.unique())

182587

In [112]:
df_clean = df_clean.drop_duplicates()

In [113]:
len(df_clean.speaker_text.unique())

182587

In [114]:
df_clean.shape

(182587, 2)

In [ ]:
GEN_BATCH    = 10   
TARGET_PER_CLASS = 35000
df_orig = df_clean
counts  = df_orig[EMOTION].value_counts().to_dict()

print("Текущее распределение:", counts)
print("Будем генерить до", TARGET_PER_CLASS, "примеров на класс")

Текущее распределение: {'neutral': 79973, 'angry': 34794, 'sad': 34294, 'positive': 33526}
Будем генерить до 35000 примеров на класс


In [ ]:

new_rows = []

for emotion, cnt in counts.items():
    need = TARGET_PER_CLASS - cnt
    if need <= 0:
        continue

    few_shot = df_orig[df_orig[EMOTION] == emotion][TEXT] \
                  .sample(3, random_state=42).tolist()

    print(f"\nКласс «{emotion}»: нужно дополнить {need} примеров")
    pbar = tqdm(total=need, desc=emotion)

    while need > 0:
        batch = min(GEN_BATCH, need)

        prompt  = f"Ниже примеры высказываний, выражающих эмоцию «{emotion}» (на русском):\n"
        for txt in few_shot:
            prompt += f"- «{txt}»\n"
        prompt += (
            f"\nПожалуйста, сгенерируй {batch} новых коротких реплик "
            "(не более 15 слов каждая) с той же эмоцией и оформи их в виде списка:\n"
        )
        for i in range(1, batch+1):
            prompt += f"{i}. \n"

        resp = client.chat.completions.create(
            model=MODEL,
            messages=[{"role":"user","content":prompt}],
        )
        text = resp.choices[0].message.content

        for line in text.splitlines():
            clean = line.lstrip('0123456789. ').strip(' «»"')
            if clean and need > 0:
                new_rows.append({TEXT: clean, EMOTION: emotion})
                need -= 1
                pbar.update(1)
                if need == 0:
                    break

    pbar.close()


Класс «angry»: нужно дополнить 206 примеров


angry: 100%|██████████| 206/206 [01:43<00:00,  1.99it/s]



Класс «sad»: нужно дополнить 706 примеров


sad: 100%|██████████| 706/706 [06:02<00:00,  1.95it/s]



Класс «positive»: нужно дополнить 1474 примеров


positive: 100%|██████████| 1474/1474 [12:36<00:00,  1.95it/s]


In [ ]:
df_new = pd.concat([df_clean, pd.DataFrame(new_rows)])
counts = df_new.groupby('speaker_text')['final_emotion'] \
                .nunique() \
                .reset_index(name='n_emotions')

good_texts = counts.loc[counts['n_emotions']==1, 'speaker_text']
df_clean = df_new[df_new['speaker_text'].isin(good_texts)].copy()

In [119]:
df_clean.shape

(184973, 2)

In [121]:
len(df_clean.speaker_text.unique())

184326

In [123]:
df_clean = df_clean.drop_duplicates()

In [124]:
df_clean.shape, len(df_clean.speaker_text.unique())

((184326, 2), 184326)

In [125]:
df_clean.final_emotion.value_counts()

final_emotion
neutral     79973
angry       34959
sad         34949
positive    34445
Name: count, dtype: int64

In [126]:
df_clean.final_emotion.value_counts(normalize=True)

final_emotion
neutral     0.433867
angry       0.189659
sad         0.189604
positive    0.186870
Name: proportion, dtype: float64

In [ ]:
GEN_BATCH    = 10    
TARGET_PER_CLASS = 40000 
df_orig = df_clean
counts  = df_orig[EMOTION].value_counts().to_dict()

print("Текущее распределение:", counts)
print("Будем генерить до", TARGET_PER_CLASS, "примеров на класс")

Текущее распределение: {'neutral': 79973, 'angry': 34959, 'sad': 34949, 'positive': 34445}
Будем генерить до 40000 примеров на класс


In [ ]:
from tqdm import tqdm
seen_texts = set(df_orig[TEXT].tolist())

new_rows = []

for emotion, cnt in counts.items():
    need = TARGET_PER_CLASS - cnt
    if need <= 0:
        continue

    few_shot = df_orig[df_orig[EMOTION] == emotion][TEXT] \
                  .sample(3, random_state=42).tolist()

    print(f"\nКласс «{emotion}»: нужно дополнить {need} примеров")
    pbar = tqdm(total=need, desc=emotion)

    while need > 0:
        batch = min(GEN_BATCH, need)

        prompt  = f"Ниже примеры высказываний, выражающих эмоцию «{emotion}» (на русском):\n"
        for txt in few_shot:
            prompt += f"- «{txt}»\n"
        prompt += (
            f"\nПожалуйста, сгенерируй {batch} новых коротких реплик "
            "(не более 15 слов каждая) с той же эмоцией и оформи их в виде списка:\n"
        )
        for i in range(1, batch+1):
            prompt += f"{i}. \n"

        resp = client.chat.completions.create(
            model=MODEL,
            messages=[{"role":"user","content":prompt}],
        )
        text = resp.choices[0].message.content

        for line in text.splitlines():
            clean = line.lstrip('0123456789. ').strip(' «»"')
            if not clean:
                continue

            if clean in seen_texts:
                continue 
            new_rows.append({TEXT: clean, EMOTION: emotion})
            seen_texts.add(clean)
            need -= 1
            pbar.update(1)

            if need == 0:
                break

    pbar.close()

df_synthetic = pd.DataFrame(new_rows)



Класс «angry»: нужно дополнить 5041 примеров


angry: 100%|██████████| 5041/5041 [1:09:02<00:00,  1.22it/s]



Класс «sad»: нужно дополнить 5051 примеров


sad: 100%|██████████| 5051/5051 [1:07:19<00:00,  1.25it/s]



Класс «positive»: нужно дополнить 5555 примеров


positive: 100%|██████████| 5555/5555 [52:23<00:00,  1.77it/s]  


In [ ]:
df_new = pd.concat([df_clean, df_synthetic])
counts = df_new.groupby('speaker_text')['final_emotion'] \
                .nunique() \
                .reset_index(name='n_emotions')

good_texts = counts.loc[counts['n_emotions']==1, 'speaker_text']
df_clean = df_new[df_new['speaker_text'].isin(good_texts)].copy()

In [130]:
df_clean = df_clean.drop_duplicates()

In [131]:
df_clean.shape, len(df_clean.speaker_text.unique())

((199973, 2), 199973)

In [132]:
df_clean.final_emotion.value_counts(normalize=True)

final_emotion
neutral     0.399919
positive    0.200027
sad         0.200027
angry       0.200027
Name: proportion, dtype: float64

In [133]:
df_clean.final_emotion.value_counts()

final_emotion
neutral     79973
positive    40000
sad         40000
angry       40000
Name: count, dtype: int64

In [ ]:
GEN_BATCH    = 10     
TARGET_PER_CLASS = 42000  
df_orig = df_clean
counts  = df_orig[EMOTION].value_counts().to_dict()

print("Текущее распределение:", counts)
print("Будем генерить до", TARGET_PER_CLASS, "примеров на класс")

Текущее распределение: {'neutral': 79973, 'positive': 40000, 'sad': 40000, 'angry': 40000}
Будем генерить до 42000 примеров на класс


In [ ]:
from tqdm import tqdm

seen_texts = set(df_orig[TEXT].tolist())

new_rows = []

for emotion, cnt in counts.items():
    need = TARGET_PER_CLASS - cnt
    if need <= 0:
        continue

    few_shot = df_orig[df_orig[EMOTION] == emotion][TEXT] \
                  .sample(3, random_state=42).tolist()

    print(f"\nКласс «{emotion}»: нужно дополнить {need} примеров")
    pbar = tqdm(total=need, desc=emotion)

    while need > 0:
        batch = min(GEN_BATCH, need)

        prompt  = f"Ниже примеры высказываний, выражающих эмоцию «{emotion}» (на русском):\n"
        for txt in few_shot:
            prompt += f"- «{txt}»\n"
        prompt += (
            f"\nПожалуйста, сгенерируй {batch} новых коротких реплик "
            "(не более 15 слов каждая) с той же эмоцией и оформи их в виде списка:\n"
        )
        for i in range(1, batch+1):
            prompt += f"{i}. \n"

        resp = client.chat.completions.create(
            model=MODEL,
            messages=[{"role":"user","content":prompt}],
        )
        text = resp.choices[0].message.content

        for line in text.splitlines():
            clean = line.lstrip('0123456789. ').strip(' «»"')
            if not clean:
                continue

            if clean in seen_texts:
                continue 
            new_rows.append({TEXT: clean, EMOTION: emotion})
            seen_texts.add(clean)
            need -= 1
            pbar.update(1)

            if need == 0:
                break

    pbar.close()

df_synthetic = pd.DataFrame(new_rows)


Класс «positive»: нужно дополнить 2000 примеров


positive: 100%|██████████| 2000/2000 [16:32<00:00,  2.01it/s]



Класс «sad»: нужно дополнить 2000 примеров


sad: 100%|██████████| 2000/2000 [24:16<00:00,  1.37it/s]  



Класс «angry»: нужно дополнить 2000 примеров


angry: 100%|██████████| 2000/2000 [14:59<00:00,  2.22it/s]


In [ ]:
df_new = pd.concat([df_clean, df_synthetic])
counts = df_new.groupby('speaker_text')['final_emotion'] \
                .nunique() \
                .reset_index(name='n_emotions')

good_texts = counts.loc[counts['n_emotions']==1, 'speaker_text']
df_clean = df_new[df_new['speaker_text'].isin(good_texts)].copy()

In [138]:
df_clean = df_clean.drop_duplicates()

In [139]:
df_clean.shape, len(df_clean.speaker_text.unique())

((205973, 2), 205973)

In [140]:
df_clean.final_emotion.value_counts(normalize=True)

final_emotion
neutral     0.388269
positive    0.203910
sad         0.203910
angry       0.203910
Name: proportion, dtype: float64

In [141]:
df_clean.final_emotion.value_counts()

final_emotion
neutral     79973
positive    42000
sad         42000
angry       42000
Name: count, dtype: int64

In [ ]:
GEN_BATCH    = 10     
TARGET_PER_CLASS = 45000  

df_orig = df_clean
counts  = df_orig[EMOTION].value_counts().to_dict()

print("Текущее распределение:", counts)
print("Будем генерить до", TARGET_PER_CLASS, "примеров на класс")

Текущее распределение: {'neutral': 79973, 'positive': 42000, 'sad': 42000, 'angry': 42000}
Будем генерить до 45000 примеров на класс


In [ ]:
from tqdm import tqdm

seen_texts = set(df_orig[TEXT].tolist())

new_rows = []

for emotion, cnt in counts.items():
    need = TARGET_PER_CLASS - cnt
    if need <= 0:
        continue

    few_shot = df_orig[df_orig[EMOTION] == emotion][TEXT] \
                  .sample(3, random_state=42).tolist()

    print(f"\nКласс «{emotion}»: нужно дополнить {need} примеров")
    pbar = tqdm(total=need, desc=emotion)

    while need > 0:
        batch = min(GEN_BATCH, need)

        prompt  = f"Ниже примеры высказываний, выражающих эмоцию «{emotion}» (на русском):\n"
        for txt in few_shot:
            prompt += f"- «{txt}»\n"
        prompt += (
            f"\nПожалуйста, сгенерируй {batch} новых коротких реплик "
            "(не более 15 слов каждая) с той же эмоцией и оформи их в виде списка:\n"
        )
        for i in range(1, batch+1):
            prompt += f"{i}. \n"

        resp = client.chat.completions.create(
            model=MODEL,
            messages=[{"role":"user","content":prompt}],
        )
        text = resp.choices[0].message.content

        for line in text.splitlines():
            clean = line.lstrip('0123456789. ').strip(' «»"')
            if not clean:
                continue

            if clean in seen_texts:
                continue  
            new_rows.append({TEXT: clean, EMOTION: emotion})
            seen_texts.add(clean)
            need -= 1
            pbar.update(1)

            if need == 0:
                break

    pbar.close()

df_synthetic = pd.DataFrame(new_rows)


Класс «positive»: нужно дополнить 3000 примеров


positive: 100%|██████████| 3000/3000 [26:47<00:00,  1.87it/s]



Класс «sad»: нужно дополнить 3000 примеров


sad: 100%|██████████| 3000/3000 [30:11<00:00,  1.66it/s]  



Класс «angry»: нужно дополнить 3000 примеров


angry: 100%|██████████| 3000/3000 [34:08<00:00,  1.46it/s]  


In [ ]:
df_new = pd.concat([df_clean, df_synthetic])
counts = df_new.groupby('speaker_text')['final_emotion'] \
                .nunique() \
                .reset_index(name='n_emotions')

good_texts = counts.loc[counts['n_emotions']==1, 'speaker_text']
df_clean = df_new[df_new['speaker_text'].isin(good_texts)].copy()

In [145]:
df_clean = df_clean.drop_duplicates()
df_clean.shape, len(df_clean.speaker_text.unique())

((214973, 2), 214973)

In [146]:
df_clean.final_emotion.value_counts(normalize=True)

final_emotion
neutral     0.372014
positive    0.209329
sad         0.209329
angry       0.209329
Name: proportion, dtype: float64

In [147]:
df_clean.final_emotion.value_counts()

final_emotion
neutral     79973
positive    45000
sad         45000
angry       45000
Name: count, dtype: int64

In [ ]:
GEN_BATCH    = 10      
TARGET_PER_CLASS = 46000 
df_orig = df_clean
counts  = df_orig[EMOTION].value_counts().to_dict()

print("Текущее распределение:", counts)
print("Будем генерить до", TARGET_PER_CLASS, "примеров на класс")

Текущее распределение: {'neutral': 79973, 'positive': 45000, 'sad': 45000, 'angry': 45000}
Будем генерить до 46000 примеров на класс


In [ ]:
from tqdm import tqdm

seen_texts = set(df_orig[TEXT].tolist())

new_rows = []

for emotion, cnt in counts.items():
    need = TARGET_PER_CLASS - cnt
    if need <= 0:
        continue

    few_shot = df_orig[df_orig[EMOTION] == emotion][TEXT] \
                  .sample(3, random_state=42).tolist()

    print(f"\nКласс «{emotion}»: нужно дополнить {need} примеров")
    pbar = tqdm(total=need, desc=emotion)

    while need > 0:
        batch = min(GEN_BATCH, need)

        prompt  = f"Ниже примеры высказываний, выражающих эмоцию «{emotion}» (на русском):\n"
        for txt in few_shot:
            prompt += f"- «{txt}»\n"
        prompt += (
            f"\nПожалуйста, сгенерируй {batch} новых коротких реплик "
            "(не более 15 слов каждая) с той же эмоцией и оформи их в виде списка:\n"
        )
        for i in range(1, batch+1):
            prompt += f"{i}. \n"

        resp = client.chat.completions.create(
            model=MODEL,
            messages=[{"role":"user","content":prompt}],
        )
        text = resp.choices[0].message.content

        for line in text.splitlines():
            clean = line.lstrip('0123456789. ').strip(' «»"')
            if not clean:
                continue

            if clean in seen_texts:
                continue  
            new_rows.append({TEXT: clean, EMOTION: emotion})
            seen_texts.add(clean)
            need -= 1
            pbar.update(1)

            if need == 0:
                break

    pbar.close()

df_synthetic = pd.DataFrame(new_rows)
df_new = pd.concat([df_clean, df_synthetic])
counts = df_new.groupby('speaker_text')['final_emotion'] \
                .nunique() \
                .reset_index(name='n_emotions')

good_texts = counts.loc[counts['n_emotions']==1, 'speaker_text']
df_clean = df_new[df_new['speaker_text'].isin(good_texts)].copy()
df_clean = df_clean.drop_duplicates()
print(df_clean.shape, len(df_clean.speaker_text.unique()))
df_clean.final_emotion.value_counts(normalize=True)


Класс «positive»: нужно дополнить 1000 примеров


positive: 100%|██████████| 1000/1000 [09:36<00:00,  1.73it/s]



Класс «sad»: нужно дополнить 1000 примеров


sad: 100%|██████████| 1000/1000 [07:05<00:00,  2.35it/s]



Класс «angry»: нужно дополнить 1000 примеров


angry: 100%|██████████| 1000/1000 [07:58<00:00,  2.09it/s]


(217973, 2) 217973


final_emotion
neutral     0.366894
positive    0.211035
sad         0.211035
angry       0.211035
Name: proportion, dtype: float64

In [150]:
df_clean.final_emotion.value_counts()

final_emotion
neutral     79973
positive    46000
sad         46000
angry       46000
Name: count, dtype: int64

In [ ]:
GEN_BATCH    = 10     
TARGET_PER_CLASS = 47000 
df_orig = df_clean
counts  = df_orig[EMOTION].value_counts().to_dict()

print("Текущее распределение:", counts)
print("Будем генерить до", TARGET_PER_CLASS, "примеров на класс")

seen_texts = set(df_orig[TEXT].tolist())

new_rows = []

for emotion, cnt in counts.items():
    need = TARGET_PER_CLASS - cnt
    if need <= 0:
        continue

    few_shot = df_orig[df_orig[EMOTION] == emotion][TEXT] \
                  .sample(3, random_state=42).tolist()

    print(f"\nКласс «{emotion}»: нужно дополнить {need} примеров")
    pbar = tqdm(total=need, desc=emotion)

    while need > 0:
        batch = min(GEN_BATCH, need)

        prompt  = f"Ниже примеры высказываний, выражающих эмоцию «{emotion}» (на русском):\n"
        for txt in few_shot:
            prompt += f"- «{txt}»\n"
        prompt += (
            f"\nПожалуйста, сгенерируй {batch} новых коротких реплик "
            "(не более 15 слов каждая) с той же эмоцией и оформи их в виде списка:\n"
        )
        for i in range(1, batch+1):
            prompt += f"{i}. \n"

        resp = client.chat.completions.create(
            model=MODEL,
            messages=[{"role":"user","content":prompt}],
        )
        text = resp.choices[0].message.content

        for line in text.splitlines():
            clean = line.lstrip('0123456789. ').strip(' «»"')
            if not clean:
                continue

            if clean in seen_texts:
                continue  
            new_rows.append({TEXT: clean, EMOTION: emotion})
            seen_texts.add(clean)
            need -= 1
            pbar.update(1)

            if need == 0:
                break

    pbar.close()

df_synthetic = pd.DataFrame(new_rows)
df_new = pd.concat([df_clean, df_synthetic])
counts = df_new.groupby('speaker_text')['final_emotion'] \
                .nunique() \
                .reset_index(name='n_emotions')

good_texts = counts.loc[counts['n_emotions']==1, 'speaker_text']
df_clean = df_new[df_new['speaker_text'].isin(good_texts)].copy()
df_clean = df_clean.drop_duplicates()
print(df_clean.shape, len(df_clean.speaker_text.unique()))
df_clean.final_emotion.value_counts(normalize=True)

Текущее распределение: {'neutral': 79973, 'positive': 46000, 'sad': 46000, 'angry': 46000}
Будем генерить до 47000 примеров на класс

Класс «positive»: нужно дополнить 1000 примеров


positive: 100%|██████████| 1000/1000 [08:53<00:00,  1.87it/s]



Класс «sad»: нужно дополнить 1000 примеров


sad: 100%|██████████| 1000/1000 [06:47<00:00,  2.46it/s]



Класс «angry»: нужно дополнить 1000 примеров


angry: 100%|██████████| 1000/1000 [18:27<00:00,  1.11s/it]


(220973, 2) 220973


final_emotion
neutral     0.361913
positive    0.212696
sad         0.212696
angry       0.212696
Name: proportion, dtype: float64

In [ ]:
GEN_BATCH    = 10      
TARGET_PER_CLASS = 48000 

df_orig = df_clean
counts  = df_orig[EMOTION].value_counts().to_dict()

print("Текущее распределение:", counts)
print("Будем генерить до", TARGET_PER_CLASS, "примеров на класс")

seen_texts = set(df_orig[TEXT].tolist())

new_rows = []

for emotion, cnt in counts.items():
    need = TARGET_PER_CLASS - cnt
    if need <= 0:
        continue

    few_shot = df_orig[df_orig[EMOTION] == emotion][TEXT] \
                  .sample(3, random_state=42).tolist()

    print(f"\nКласс «{emotion}»: нужно дополнить {need} примеров")
    pbar = tqdm(total=need, desc=emotion)

    while need > 0:
        batch = min(GEN_BATCH, need)

        prompt  = f"Ниже примеры высказываний, выражающих эмоцию «{emotion}» (на русском):\n"
        for txt in few_shot:
            prompt += f"- «{txt}»\n"
        prompt += (
            f"\nПожалуйста, сгенерируй {batch} новых коротких реплик "
            "(не более 15 слов каждая) с той же эмоцией и оформи их в виде списка:\n"
        )
        for i in range(1, batch+1):
            prompt += f"{i}. \n"

        resp = client.chat.completions.create(
            model=MODEL,
            messages=[{"role":"user","content":prompt}],
        )
        text = resp.choices[0].message.content

        for line in text.splitlines():
            clean = line.lstrip('0123456789. ').strip(' «»"')
            if not clean:
                continue

            if clean in seen_texts:
                continue 
            new_rows.append({TEXT: clean, EMOTION: emotion})
            seen_texts.add(clean)
            need -= 1
            pbar.update(1)

            if need == 0:
                break

    pbar.close()

df_synthetic = pd.DataFrame(new_rows)
df_new = pd.concat([df_clean, df_synthetic])
counts = df_new.groupby('speaker_text')['final_emotion'] \
                .nunique() \
                .reset_index(name='n_emotions')

good_texts = counts.loc[counts['n_emotions']==1, 'speaker_text']
df_clean = df_new[df_new['speaker_text'].isin(good_texts)].copy()
df_clean = df_clean.drop_duplicates()
print(df_clean.shape, len(df_clean.speaker_text.unique()))
df_clean.final_emotion.value_counts(normalize=True)

In [ ]:
df_synthetic = pd.DataFrame(new_rows)
df_new = pd.concat([df_clean, df_synthetic])
counts = df_new.groupby('speaker_text')['final_emotion'] \
                .nunique() \
                .reset_index(name='n_emotions')

good_texts = counts.loc[counts['n_emotions']==1, 'speaker_text']
df_clean = df_new[df_new['speaker_text'].isin(good_texts)].copy()
df_clean = df_clean.drop_duplicates()
print(df_clean.shape, len(df_clean.speaker_text.unique()))
df_clean.final_emotion.value_counts(normalize=True)

(223460, 2) 223460


final_emotion
neutral     0.357885
positive    0.214804
sad         0.214804
angry       0.212508
Name: proportion, dtype: float64

In [154]:
df_clean.final_emotion.value_counts()

final_emotion
neutral     79973
positive    48000
sad         48000
angry       47487
Name: count, dtype: int64

In [ ]:
GEN_BATCH    = 10     
TARGET_PER_CLASS = 48000 
df_orig = df_clean
counts  = df_orig[EMOTION].value_counts().to_dict()

print("Текущее распределение:", counts)
print("Будем генерить до", TARGET_PER_CLASS, "примеров на класс")

seen_texts = set(df_orig[TEXT].tolist())

new_rows = []

for emotion, cnt in counts.items():
    need = TARGET_PER_CLASS - cnt
    if need <= 0:
        continue

    few_shot = df_orig[df_orig[EMOTION] == emotion][TEXT] \
                  .sample(3, random_state=42).tolist()

    print(f"\nКласс «{emotion}»: нужно дополнить {need} примеров")
    pbar = tqdm(total=need, desc=emotion)

    while need > 0:
        batch = min(GEN_BATCH, need)

        prompt  = f"Ниже примеры высказываний, выражающих эмоцию «{emotion}» (на русском):\n"
        for txt in few_shot:
            prompt += f"- «{txt}»\n"
        prompt += (
            f"\nПожалуйста, сгенерируй {batch} новых коротких реплик "
            "(не более 15 слов каждая) с той же эмоцией и оформи их в виде списка:\n"
        )
        for i in range(1, batch+1):
            prompt += f"{i}. \n"

        resp = client.chat.completions.create(
            model=MODEL,
            messages=[{"role":"user","content":prompt}],
        )
        text = resp.choices[0].message.content

        for line in text.splitlines():
            clean = line.lstrip('0123456789. ').strip(' «»"')
            if not clean:
                continue

            if clean in seen_texts:
                continue  
            new_rows.append({TEXT: clean, EMOTION: emotion})
            seen_texts.add(clean)
            need -= 1
            pbar.update(1)

            if need == 0:
                break

    pbar.close()

df_synthetic = pd.DataFrame(new_rows)
df_new = pd.concat([df_clean, df_synthetic])
counts = df_new.groupby('speaker_text')['final_emotion'] \
                .nunique() \
                .reset_index(name='n_emotions')

good_texts = counts.loc[counts['n_emotions']==1, 'speaker_text']
df_clean = df_new[df_new['speaker_text'].isin(good_texts)].copy()
df_clean = df_clean.drop_duplicates()
print(df_clean.shape, len(df_clean.speaker_text.unique()))
df_clean.final_emotion.value_counts(normalize=True)

Текущее распределение: {'neutral': 79973, 'positive': 48000, 'sad': 48000, 'angry': 47487}
Будем генерить до 48000 примеров на класс

Класс «angry»: нужно дополнить 513 примеров


angry: 100%|██████████| 513/513 [03:40<00:00,  2.32it/s]


(223973, 2) 223973


final_emotion
neutral     0.357065
positive    0.214312
sad         0.214312
angry       0.214312
Name: proportion, dtype: float64

In [156]:
df_clean.final_emotion.value_counts()

final_emotion
neutral     79973
positive    48000
sad         48000
angry       48000
Name: count, dtype: int64

In [157]:
df_clean.tail(10)

,speaker_text,final_emotion
503,"Не надо меня бесить, я не в настроении!",angry
504,Сколько можно тратить время впустую?!,angry
505,Ты снова это делаешь?! Это неприемлемо!,angry
506,"Не заставляй меня злиться, я не шучу!",angry
507,"Ты явно не понимаешь, как меня это бесит!",angry
508,"Ты меня бесишь, когда так себя ведёшь!",angry
509,"Я это больше не потерплю, уяснил?.",angry
510,"Достало постоянно тебя уговаривать, действуй уже!",angry
511,"Хватит ссориться, мне это надоело до чертиков!",angry
512,"Мне это надоело, ушел бы ты с глаз долой!",angry


In [158]:
df_clean.to_csv('C://Users//a.karakuchukova//Desktop//Университет//Диплом//DataSets//dusha.crowd//gpt_oversampled_dusha_crowd.csv')

In [ ]:
GEN_PER_ITER = 5  
MAX_ITERS    = 10  

df = pd.read_csv(INPUT_CSV)
counts = df[EMOTION].value_counts()
target = counts.max()
print("Текущий максимум:", target)

new_rows = []

Текущий максимум: 87574


In [5]:
df = pd.read_csv('C://Users//a.karakuchukova//Desktop//Университет//Диплом//DataSets//dusha.crowd//gpt_oversampled_dusha_crowd.csv')

In [7]:
df.final_emotion.value_counts()

final_emotion
neutral     79973
positive    48000
sad         48000
angry       48000
Name: count, dtype: int64

In [ ]:
GEN_BATCH    = 10     
TARGET_PER_CLASS = 55000 
df_clean=df
df_orig = df_clean
counts  = df_orig[EMOTION].value_counts().to_dict()

print("Текущее распределение:", counts)
print("Будем генерить до", TARGET_PER_CLASS, "примеров на класс")

seen_texts = set(df_orig[TEXT].tolist())

new_rows = []

for emotion, cnt in counts.items():
    need = TARGET_PER_CLASS - cnt
    if need <= 0:
        continue

    few_shot = df_orig[df_orig[EMOTION] == emotion][TEXT] \
                  .sample(3, random_state=42).tolist()

    print(f"\nКласс «{emotion}»: нужно дополнить {need} примеров")
    pbar = tqdm(total=need, desc=emotion)

    while need > 0:
        batch = min(GEN_BATCH, need)

        prompt  = f"Ниже примеры высказываний, выражающих эмоцию «{emotion}» (на русском):\n"
        for txt in few_shot:
            prompt += f"- «{txt}»\n"
        prompt += (
            f"\nПожалуйста, сгенерируй {batch} новых коротких реплик "
            "(не более 15 слов каждая) с той же эмоцией и оформи их в виде списка:\n"
        )
        for i in range(1, batch+1):
            prompt += f"{i}. \n"

        resp = client.chat.completions.create(
            model=MODEL,
            messages=[{"role":"user","content":prompt}],
        )
        text = resp.choices[0].message.content

        for line in text.splitlines():
            clean = line.lstrip('0123456789. ').strip(' «»"')
            if not clean:
                continue

            if clean in seen_texts:
                continue 
            new_rows.append({TEXT: clean, EMOTION: emotion})
            seen_texts.add(clean)
            need -= 1
            pbar.update(1)

            if need == 0:
                break

    pbar.close()

df_synthetic = pd.DataFrame(new_rows)
df_new = pd.concat([df_clean, df_synthetic])
counts = df_new.groupby('speaker_text')['final_emotion'] \
                .nunique() \
                .reset_index(name='n_emotions')

good_texts = counts.loc[counts['n_emotions']==1, 'speaker_text']
df_clean = df_new[df_new['speaker_text'].isin(good_texts)].copy()
df_clean = df_clean.drop_duplicates()
print(df_clean.shape, len(df_clean.speaker_text.unique()))
df_clean.final_emotion.value_counts(normalize=True)

Текущее распределение: {'neutral': 79973, 'positive': 48000, 'sad': 48000, 'angry': 48000}
Будем генерить до 55000 примеров на класс

Класс «positive»: нужно дополнить 7000 примеров


positive: 100%|██████████| 7000/7000 [1:08:19<00:00,  1.71it/s]  



Класс «sad»: нужно дополнить 7000 примеров


sad: 100%|██████████| 7000/7000 [51:57<00:00,  2.25it/s]



Класс «angry»: нужно дополнить 7000 примеров


angry: 100%|██████████| 7000/7000 [1:28:02<00:00,  1.33it/s] 


(244973, 3) 244973


final_emotion
neutral     0.326456
positive    0.224515
sad         0.224515
angry       0.224515
Name: proportion, dtype: float64

In [15]:
df_synthetic.final_emotion.value_counts()

final_emotion
positive    7000
sad         7000
angry       7000
Name: count, dtype: int64

In [16]:
df_new = pd.concat([df_clean, df_synthetic])
df_new.final_emotion.value_counts()

final_emotion
neutral     79973
positive    55000
sad         55000
angry       55000
Name: count, dtype: int64

In [19]:
df_clean = df_new.drop_duplicates()
df_clean.final_emotion.value_counts()

final_emotion
neutral     79973
positive    55000
sad         55000
angry       55000
Name: count, dtype: int64

In [21]:
print(df_clean.shape, len(df_clean.speaker_text.unique()))

(244973, 3) 244973


In [ ]:
GEN_BATCH    = 10     
TARGET_PER_CLASS = 60000 

df_orig = df_clean
counts  = df_orig[EMOTION].value_counts().to_dict()

print("Текущее распределение:", counts)
print("Будем генерить до", TARGET_PER_CLASS, "примеров на класс")

seen_texts = set(df_orig[TEXT].tolist())

new_rows = []

for emotion, cnt in counts.items():
    need = TARGET_PER_CLASS - cnt
    if need <= 0:
        continue

    few_shot = df_orig[df_orig[EMOTION] == emotion][TEXT] \
                  .sample(3, random_state=42).tolist()

    print(f"\nКласс «{emotion}»: нужно дополнить {need} примеров")
    pbar = tqdm(total=need, desc=emotion)

    while need > 0:
        batch = min(GEN_BATCH, need)

        prompt  = f"Ниже примеры высказываний, выражающих эмоцию «{emotion}» (на русском):\n"
        for txt in few_shot:
            prompt += f"- «{txt}»\n"
        prompt += (
            f"\nПожалуйста, сгенерируй {batch} новых коротких реплик "
            "(не более 15 слов каждая) с той же эмоцией и оформи их в виде списка:\n"
        )
        for i in range(1, batch+1):
            prompt += f"{i}. \n"

        resp = client.chat.completions.create(
            model=MODEL,
            messages=[{"role":"user","content":prompt}],
        )
        text = resp.choices[0].message.content

        for line in text.splitlines():
            clean = line.lstrip('0123456789. ').strip(' «»"')
            if not clean:
                continue

            if clean in seen_texts:
                continue 
            new_rows.append({TEXT: clean, EMOTION: emotion})
            seen_texts.add(clean)
            need -= 1
            pbar.update(1)

            if need == 0:
                break

    pbar.close()

df_synthetic = pd.DataFrame(new_rows)
df_new = pd.concat([df_clean, df_synthetic])
counts = df_new.groupby('speaker_text')['final_emotion'] \
                .nunique() \
                .reset_index(name='n_emotions')

good_texts = counts.loc[counts['n_emotions']==1, 'speaker_text']
df_clean = df_new[df_new['speaker_text'].isin(good_texts)].copy()
df_clean = df_clean.drop_duplicates()
print(df_clean.shape, len(df_clean.speaker_text.unique()))
df_clean.final_emotion.value_counts(normalize=True)

Текущее распределение: {'neutral': 79973, 'positive': 55000, 'sad': 55000, 'angry': 55000}
Будем генерить до 60000 примеров на класс

Класс «positive»: нужно дополнить 5000 примеров


positive: 100%|██████████| 5000/5000 [57:51<00:00,  1.44it/s]



Класс «sad»: нужно дополнить 5000 примеров


sad: 100%|██████████| 5000/5000 [56:58<00:00,  1.46it/s]  



Класс «angry»: нужно дополнить 5000 примеров


angry: 100%|██████████| 5000/5000 [54:28<00:00,  1.53it/s]   


(259973, 3) 259973


final_emotion
neutral     0.307620
positive    0.230793
sad         0.230793
angry       0.230793
Name: proportion, dtype: float64

In [23]:
df_clean.final_emotion.value_counts()

final_emotion
neutral     79973
positive    60000
sad         60000
angry       60000
Name: count, dtype: int64

In [25]:
df_clean.to_csv('C://Users//a.karakuchukova//Desktop//Университет//Диплом//DataSets//dusha.crowd//gpt_oversampled_dusha_crowd.csv')